<a href="https://colab.research.google.com/github/GJHSimmons/personal/blob/main/1D_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Parameters
num_agents = 100
space_size = 100
num_steps = 100
diffusion_coefficient = 0.5  # D ∈ [0, 1], higher D means more movement

# Initialize positions
positions = [50 for _ in range(num_agents)]
history = [positions.copy()]

for step in range(num_steps):
    new_positions = []
    for pos in positions:
        if random.random() < diffusion_coefficient:
            move = random.choice([-1, 1])  # Move left or right
            new_pos = pos + move
            new_pos = max(0, min(space_size - 1, new_pos))
        else:
            new_pos = pos  # Stay
        new_positions.append(new_pos)
    positions = new_positions
    history.append(positions.copy())


import plotly.express as px

data = []
for time_step, pos_list in enumerate(history):
    counts = [0] * space_size
    for pos in pos_list:
        counts[pos] += 1
    for pos, count in enumerate(counts):
        data.append({"Time": time_step, "Position": pos, "Count": count})

df_long = pd.DataFrame(data)

# --- Plotly Express Animation ---
fig = px.bar(
    df_long,
    x="Position",
    y="Count",
    animation_frame="Time",
    range_y=[0, max(df_long["Count"]) + 5],
    title="1D Agent-Based Diffusion (Animated)",
    labels={"Count": "Number of Agents"},
)

fig.update_layout(
    xaxis=dict(dtick=10),
    transition={'duration': 100},
)
fig.show()

In [ ]:
# Match ABM parameters
space_size = 100
num_steps = 100
D = 0.5  # Diffusion coefficient (must match ABM)
dx = 1
dt = 1

# Initial condition: delta function in center
u = np.zeros(space_size)
u[space_size // 2] = 100  # 100% of mass at center

# Track the solution at each time step
pde_history = [u.copy()]

# Explicit finite difference solution
for _ in range(num_steps):
    u_new = u.copy()
    for i in range(1, space_size - 1):
        u_new[i] = u[i] + D * dt / dx**2 * (u[i+1] - 2*u[i] + u[i-1])
    u = u_new
    pde_history.append(u.copy())

# Convert to DataFrame (long format for Plotly comparison)
pde_data = []
for t, row in enumerate(pde_history):
    for x, value in enumerate(row):
        pde_data.append({"Time": t, "Position": x, "Density": value})
df_pde = pd.DataFrame(pde_data)


fig = px.bar(
    df_pde,
    x="Position",
    y="Density",
    animation_frame="Time",
    title="PDE Model of 1D Diffusion (Animated)",
    range_y=[0, max(df_pde['Density']) + 0.01]
)
fig.update_layout(transition={'duration': 100})
fig.show()